In [ ]:
!pip3 install tensorflow

In [3]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from string import punctuation

In [4]:
import requests
content = requests.get("http://www.gutenberg.org/cache/epub/11/pg11.txt").text

In [ ]:
open("wonderland.txt", "w", encoding="utf-8").write(content)

In [6]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30

In [7]:
FILE_PATH = "/content/wonderland.txt"
text = open(FILE_PATH, encoding="utf-8").read()


In [9]:
text=text.lower()

In [10]:
text = text.translate(str.maketrans("", "", punctuation))

In [11]:
n_chars = len(text)
n_chars

158225

In [41]:
vocab=''.join(sorted(set(text)))
vocab

'\n 0123456789abcdefghijklmnopqrstuvwxyzù—‘’“”•™\ufeff'

In [13]:
n_unique_chars = len(vocab)
n_unique_chars

47

In [14]:
# dictionary that converts characters to integers
char2int ={c:i for i ,c in enumerate (vocab)}
char2int

{'\n': 0,
 ' ': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37,
 'ù': 38,
 '—': 39,
 '‘': 40,
 '’': 41,
 '“': 42,
 '”': 43,
 '•': 44,
 '™': 45,
 '\ufeff': 46}

In [15]:
# dictionary that converts integers to characters
int2char={i : c for i ,c in enumerate (vocab)}
int2char

{0: '\n',
 1: ' ',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9',
 12: 'a',
 13: 'b',
 14: 'c',
 15: 'd',
 16: 'e',
 17: 'f',
 18: 'g',
 19: 'h',
 20: 'i',
 21: 'j',
 22: 'k',
 23: 'l',
 24: 'm',
 25: 'n',
 26: 'o',
 27: 'p',
 28: 'q',
 29: 'r',
 30: 's',
 31: 't',
 32: 'u',
 33: 'v',
 34: 'w',
 35: 'x',
 36: 'y',
 37: 'z',
 38: 'ù',
 39: '—',
 40: '‘',
 41: '’',
 42: '“',
 43: '”',
 44: '•',
 45: '™',
 46: '\ufeff'}

In [16]:
# convert all text into integers
encoded_text=np.array([char2int[c]for c in text])
encoded_text

array([46, 31, 19, ...,  0,  0,  0])

In [18]:
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
char_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [19]:
# print first 5 characters
for char in char_dataset.take(8):
  print(char.numpy(),int2char[char.numpy()])

46 ﻿
31 t
19 h
16 e
1  
27 p
29 r
26 o


In [ ]:
sequence_length = 100
#تقطيع النص كله الي بيانات مقطعة
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)
sequences

In [21]:
for sequence in sequences.take(3):
  print(''.join([int2char[i] for i in sequence.numpy()]))



﻿the project gutenberg ebook of alices adventures in wonderland
    
this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no re
strictions
whatsoever you may copy it give it away or reuse it under the terms
of the project gutenberg license included with this ebook or online
at wwwgutenbergorg if you are not located in the unite
d states
you will have to check the laws of the country where you are located
before using this ebook

title alices adventures in wonderland


author lewis carroll

release date june 27 2008 ebook 11
 


In [22]:
def split_sample(sample):
      ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
      for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
      return ds
# prepare inputs and targets
dataset = sequences.flat_map(split_sample)




In [24]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)


dataset = dataset.map(one_hot_samples)

In [ ]:
dataset

In [26]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿the project gutenberg ebook of alices adventures in wonderland
    
this ebook is for the use of an
Target: y
Input shape: (100, 47)
Target shape: (47,)

Input: the project gutenberg ebook of alices adventures in wonderland
    
this ebook is for the use of any
Target: o
Input shape: (100, 47)
Target shape: (47,)



In [27]:
#عمل خلط للبيانات و تقسيمها الي باشتات
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [30]:
#بناء الموديل
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=10)


Epoch 1/10
1235/1235 [==============================] - 1531s 1s/step - loss: 1.8008
Epoch 2/10
1235/1235 [==============================] - 1519s 1s/step - loss: 1.5839
Epoch 3/10
1235/1235 [==============================] - 1525s 1s/step - loss: 1.4250
Epoch 4/10
1235/1235 [==============================] - 1516s 1s/step - loss: 1.2912
Epoch 5/10
1235/1235 [==============================] - 1516s 1s/step - loss: 1.1762
Epoch 6/10
1235/1235 [==============================] - 1517s 1s/step - loss: 1.0719
Epoch 7/10
1235/1235 [==============================] - 1500s 1s/step - loss: 0.9810
Epoch 8/10
1235/1235 [==============================] - 1498s 1s/step - loss: 0.8982
Epoch 9/10
1235/1235 [==============================] - 1503s 1s/step - loss: 0.8231
Epoch 10/10
1235/1235 [==============================] - 1507s 1s/step - loss: 0.7618


In [ ]:
!pip install "git+https://github.com/tqdm/tqdm.git@devel#egg=tqdm"


In [44]:
import tqdm
seed = "chapter xiii"
vocab_size = len(char2int)
s = seed
n_chars = 10000
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 10000/10000 [10:16<00:00, 16.21it/s]

Seed: chapter xiii
Generated text:
d to grin” said the whole party go do little thing” she said to get through they were indeed you should think” and she went of the wides—” she was getting of copyright law do i think—” thought she knever my deep with the books to get thank i should think” and she went of the wides—” she was getting of copyright law do i think—” thought she knever my deep with the books to get thank i should think” and she went of the wides—” she was getting of copyright law do i think—” thought she knever my deep with the books to get thank i should think” and she went of the wides—” she was getting of copyright law do i think—” thought she knever my deep with the books to get thank i should think” and she went of the wides—” she was getting of copyright law do i think—” thought she knever my deep with the books to get thank i should think” and she went of the wides—” she was getting of copyright law do i think—” thought she knever my deep with the books to get thank 